# 02. 개발 환경

<br>

## Django REST Framework
- 가상환경 내

```bash
$ pip install djangorestframework django-cors-headers
```

<br>

- 프로젝트 및 애플리케이션 생성

```bash
$ django-admin startproject backend
$ python manage.py startapp app0
```

<br>

- Django REST Framework를 Django 프로젝트에 통합
    - `settings.py`에서
  
```python
INSTALLED_APPS = [
    # ...
    'rest_framework',
    'app0',
    # ...
]
```

<br>

- `urls.py`

```python
from django.contrib

import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('app0.urls')),
]
```

<br>

- `settings.py` 파일에 REST_FRAMEWORK 사전을 추가하여 Django REST Framework의 다양한 설정을 구성할
- 이러한 설정은 기본 인증 및 권한 클래스, 페이지네이션 설정 등을 포함가능
  - 사용 가능한 설정의 상세 목록은 공식 [Django REST Framework](https://www.django-rest-framework.org/) 문서에서 확인가능
  
- 기본 인증 및 권한 클래스를 설정

<br>

```python
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework.authentication.SessionAuthentication',
        'rest_framework.authentication.BasicAuthentication'
    ],
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ]
}
```

<br>

#### REST API 생성

- `app0/views.py`


```python
from rest_framework.decorators import api_view
from rest_framework.response import Response

@api_view(['GET'])
def hello_world(request):
    return Response('Hello, World!')
```

<br>

- `app0/urls.py`

```python
from django.urls import path
from . import views

urlpatterns = [
    path('hello/', views.hello_world),
]
```


<br>

- 이후 http://127.0.0.1:8000/api/hello/ 접속

<img src='https://wikidocs.net/images/page/197539/2_1_5.png' width=600>

<br>

### Django CORS 설정
- 프론트엔드가 백엔드에 API 요청을 보낼 때, 이는 Cross-Origin 요청으로 간주
  
  (http://localhost:3000) $\rightarrow$ (http://127.0.0.1:8000)

- **웹 브라우저는 보안을 위해 기본적으로 Cross-Origin 요청을 차단하며, 따라서 백엔드를 구성하여 프론트엔드가 요청을 보낼 수 있도록 해야 함**

<br>

#### `settings.py` 설정
- `INSTALLED_APPS` 목록에 `corsheaders`를 추가

```python
INSTALLED_APPS = [
    ...
    'corsheaders',
]
```

<br>

- `MIDDLEWARE` 목록 맨 처음에 `corsheaders.middleware.CorsMiddleware`를 추가

```python
MIDDLEWARE = [
    'corsheaders.middleware.CorsMiddleware',
    ...
]
```

<br>

- `CORS_ORIGIN_WHITELIST`와 `CORS_ALLOWED_ORIGINS`를 통하여 URL에서의 요청을 허용

```python
CORS_ORIGIN_WHITELIST = [
    'http://localhost:3000',
]

CORS_ALLOWED_ORIGINS = [
    'http://localhost:3000',
]
```

<br>

- `CORS_ORIGIN_ALLOW_ALL`를 `True`로 설정하면 모든 출처에서의 요청을 허용
- `CORS_ALLOWED_CREDENTIALS`를 `True`로 설정하면 자격 증명을 허용

```python
CORS_ORIGIN_ALLOW_ALL = True
CORS_ALLOWED_CREDENTIALS = True
```

<br>

#### 데이터베이스 마이그레이션 후 서버 실행

```bash
$ python manage.py migrate
$ python manage.py runserver
```

<br>

### Django REST Framework 주요 구성 요소

<br>

#### 시리얼라이저(Serializers)
- **시리얼라이저는 Django 모델과 같은 복잡한 데이터 유형을 JSON, XML 또는 기타 콘텐츠 유형으로 변환하여 응답으로 렌더링하거나 클라이언트가 이해할 수 있는 형식으로 데이터를 파싱하는 기능을 제공**
- 시리얼라이저는 들어오는 데이터를 파싱하고 복잡한 데이터 유형으로 다시 변환하는 역 과정도 수행

```python
from rest_framework import serializers

class BookSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100)
    author = serializers.CharField(max_length=100)
    publication_date = serializers.DateField()

    def create(self, validated_data):
        return Book.objects.create(**validated_data)

```

<br>

#### 뷰(Views)
- Django REST Framework에서 뷰는 HTTP 요청을 처리하고 HTTP 응답을 반환하는 역할을 담당
- 뷰는 종종 시리얼라이저를 사용하여 클라이언트가 이해할 수 있는 형식으로 데이터를 반환
- Django REST Framework는 APIView, GenericAPIView, ViewSet과 같은 여러 내장 뷰를 제공하여 일반적인 작업을 처리하여 API 구축을 용이하게

```python
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status

class BookListView(APIView):
    def get(self, request):
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data)

    def post(self, request):
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
``` 

<br>

#### 라우터(Routers)
- Django REST Framework의 라우터는 API 뷰에 대한 URL 패턴을 자동으로 생성
- 라우터는 API 엔드포인트를 조직하는 간단하고 일관된 방법을 제공하며 API의 버전을 관리하거나 네임스페이스 접두사를 적용하기 쉽게

```python
from rest_framework import routers
from .views import BookViewSet

router = routers.DefaultRouter()
router.register(r'books', BookViewSet)
```


<br>

#### 인증(Authentication)
- Django REST Framework는 토큰 기반 인증, 세션 인증, OAuth2와 같은 다양한 인증 메커니즘을 지원하여 인증된 사용자만 API에 액세스할 수 있도록 보장

```python
from rest_framework.authentication import TokenAuthentication
from rest_framework.permissions import IsAuthenticated

class BookListView(APIView):
    authentication_classes = [TokenAuthentication]
    permission_classes = [IsAuthenticated]

    # 뷰 코드...
```

<br>

#### 권한(Permissions)
- **Django REST Framework의 권한은 API 리소스에서 특정 작업을 수행할 수 있는 사용자를 제어**
- 내장 권한 클래스를 사용하거나 사용자 정의 권한 클래스를 생성하여 세밀한 액세스 제어를 강제
  
```python
from rest_framework.permissions import IsAdminUser

class BookListView(APIView):
    permission_classes = [IsAdminUser]

    # 뷰 코드...

```

<br>

#### 쓰로틀링(Throttling)
- **쓰로틀링은 Django REST Framework의 중요한 기능으로, API를 남용하거나 과도하게 사용하는 것으로부터 보호**
  - **클라이언트가 특정 시간 프레임 내에서 요청할 수 있는 횟수를 제한**
- Django REST Framework는 여러 내장 쓰로틀링 클래스를 제공하며 사용자 정의 쓰로틀링 클래스를 만들어 요구 사항에 맞게 설정

```python
from rest_framework.throttling import UserRateThrottle

class BookListView(APIView):
    throttle_classes = [UserRateThrottle]

    # 뷰 코드...

```

<br>

#### 페이지네이션(Pagination)
- 페이지네이션은 대량의 데이터 집합을 관리하고 API의 성능을 향상시키는 데 중요
- Django REST Framework는 limit/offset 페이지네이션, 커서 페이지네이션 등 다양한 페이지네이션 스타일을 지원하며 전역적으로 또는 뷰별로 페이지네이션 설정을 구성

```python
from rest_framework.pagination import PageNumberPagination

class CustomPagination(PageNumberPagination):
    page_size = 10
    page_size_query_param = 'page_size'
    max_page_size = 100

class BookListView(APIView):
    pagination_class = CustomPagination

    # 뷰 코드...

```

<br>

#### 필터(Filtering)
- 필터링은 API에서 반환되는 데이터를 검색, 정렬 및 필터링하는 데 도움을 주는 기능
- Django REST Framework는 Django Filter와 같은 인기있는 필터링 라이브러리와 통합되며 쿼리 매개변수 필터링 및 결과 정렬에 대한 내장 지원을 제공

```python
from django_filters.rest_framework import DjangoFilterBackend

class BookListView(APIView):
    filter_backends = [DjangoFilterBackend]
    filterset_fields = ['title', 'author']

    # 뷰 코드...

```

<br>

#### 예외(Exceptions)
- Django REST Framework에는 유효성 검사 오류, 인증 실패, 권한 거부 등과 같은 일반적인 API 오류를 처리하는 일련의 내장 예외 클래스가 포함
- 이러한 예외는 적절한 HTTP 상태 코드를 반환하며 더 자세한 오류 메시지를 반환할 수 있도록 사용자 정의

```python
from rest_framework.views import exception_handler

def custom_exception_handler(exc, context):
    response = exception_handler(exc, context)
    if response is not None:
        custom_data = {'detail': response.data['detail']}
        response.data = custom_data
    return response

```



<br>

#### 문서화(Documentation)
- 잘 문서화된 API는 개발자와 사용자 모두에게 중요
  
  Django REST Framework는 OpenAPI(이전 Swagger로 알려짐) 표준 지원 및 브라우저에서 작동하는 API 문서 생성 및 유지를 위한 여러 도구를 제공
- 가장 일반적인 방법은 `drf-yasg` 라이브러리를 사용하여 OpenAPI 스펙을 생성하는 것

```python
from rest_framework import routers
from drf_yasg import openapi
from drf_yasg.views import get_schema_view

schema_view = get_schema_view(
    openapi.Info(
        title="Book API",
        default_version='v1',
        description="API for managing books",
        terms_of_service="https://www.example.com/terms/",
        contact=openapi.Contact(email="contact@example.com"),
        license=openapi.License(name="BSD License"),
    ),
    public=True

,
    permission_classes=(permissions.AllowAny,),
)

urlpatterns = [
    # 기존 URL 패턴들...
    path('api/docs/', schema_view.with_ui('swagger', cache_timeout=0), name='schema-swagger-ui'),
    path('api/redoc/', schema_view.with_ui('redoc', cache_timeout=0), name='schema-redoc'),
]
```

<br>

<hr>